In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import re
from time import sleep
import random
#return a randomly generated number
#from a normal distribution
#but within a range
#source: https://stackoverflow.com/a/26920781
def trunc_gauss(mu, sigma, bottom, top):
    a = random.gauss(mu,sigma)
    while (bottom <= a <= top) == False:
        a = random.gauss(mu,sigma)
    return a

caps = DesiredCapabilities().FIREFOX

#function to sleep a random amount of time, mean 2.5, sd 0.5, but no less than 2.0001 or more than 2.9999 seconds
#this is necessary because if the wait time is too high, the connection between selenium and the browser might break
def short_sleep():
    sleep(trunc_gauss(2.5, 0.5, 2.0001, 2.9999))

site = "https://www.cityofdenhamsprings.com/"

driver = webdriver.Firefox(capabilities=caps)
driver.get(site)
elems = driver.find_elements_by_xpath("//a[@href]")
hrefs = []
for elem in elems:
    hrefs.append(elem.get_attribute("href"))

In [4]:
import tldextract
#check the domain name of the website's homepage
site_tld = tldextract.extract(site)

In [5]:
#check each of the urls found on the site for whether it actually belongs to the same domain
#gather the ones that do in the site_links list
site_links = []
for href in hrefs:
    href_tld = tldextract.extract(href)
    if href_tld.domain == site_tld.domain:
        site_links.append(href)

In [6]:
#remove duplicates; requires python 3.7
site_links = list(dict.fromkeys(site_links))

In [7]:
#kick out email addresses
site_links = [i for i in site_links if not '@' in i]
#which URLs have we already visited?
urls_visited = dict.fromkeys(site_links, False)

In [8]:
new_links = []

#keep going over the dict until all links have been explored
while not all(value == True for value in urls_visited.values()):

    for i in urls_visited:

        if urls_visited[i]==False:

            #go to the subdomain url
            #and extract all the urls again
            driver.get(i)
            elems = driver.find_elements_by_xpath("//a[@href]")
            hrefs = []
            for elem in elems:
                hrefs.append(elem.get_attribute("href"))
            #sub_site_links = []
            for href in hrefs:
                href_tld = tldextract.extract(href)
                if href_tld.domain == site_tld.domain:
                    new_links.append(href)

            #note that we have visited the url        
            urls_visited[i] = True

            #add all the new links
            #for link in sub_site_links:
            #    urls_visited.setdefault(link, 'False')

    #remove duplicates
    new_links = list(dict.fromkeys(new_links))
    #remove email addresses
    new_links = [i for i in new_links if not '@' in i]

    for i in new_links:
        #potentially remove a # at the end of the url (preventing duplicates)
        #and then add to the dictionary
        urls_visited.setdefault(re.sub('#$', '', i), 'False')

KeyboardInterrupt: 

In [9]:
urls_visited

{'http://www.cityofdenhamsprings.com/': 'False',
 'http://www.cityofdenhamsprings.com/purchasing.shtml': 'False',
 'https://www.cityofdenhamsprings.com/': 'False',
 'https://www.cityofdenhamsprings.com/#': True,
 'https://www.cityofdenhamsprings.com/administration.html': True,
 'https://www.cityofdenhamsprings.com/agendas---minutes-2016.html': True,
 'https://www.cityofdenhamsprings.com/agendas---minutes-2017.html': True,
 'https://www.cityofdenhamsprings.com/agendas---minutes-2018.html': True,
 'https://www.cityofdenhamsprings.com/agendas-minutes.html': True,
 'https://www.cityofdenhamsprings.com/animal-control-center.html': True,
 'https://www.cityofdenhamsprings.com/business-licenses.html': True,
 'https://www.cityofdenhamsprings.com/businesses.html': True,
 'https://www.cityofdenhamsprings.com/cemetery-services.html': True,
 'https://www.cityofdenhamsprings.com/city-calendar.html': True,
 'https://www.cityofdenhamsprings.com/city-council.html': True,
 'https://www.cityofdenhamsprin